In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
import scipy
from sklearn.linear_model import LogisticRegression
import optuna

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
# !pip install optuna

Credits to @Ants / Notebook : https://www.kaggle.com/superant/oh-my-cat

# Load DataSet

In [8]:
train = pd.read_csv('input/train.csv', index_col='id')
test = pd.read_csv('input/test.csv', index_col='id')
submission = pd.read_csv('input/sample_submission.csv', index_col='id')

Ytrain = train['target']
train = train[list(test)]
all_data=pd.concat((train, test))
print(train.shape, test.shape, all_data.shape)

(300000, 23) (200000, 23) (500000, 23)


In [10]:
encoded=pd.get_dummies(all_data, columns=all_data.columns, sparse=True)
encoded=encoded.sparse.to_coo()
encoded=encoded.tocsr()


<500000x16552 sparse matrix of type '<class 'numpy.uint8'>'
	with 11500000 stored elements in Compressed Sparse Row format>

In [11]:
Xtrain=encoded[:len(train)]
Xtest=encoded[len(train):]


In [12]:
kf=StratifiedKFold(n_splits=10)

In [13]:
def objective(trial):
    C=trial.suggest_loguniform('C', 10e-10, 10)
    model=LogisticRegression(C=C, class_weight='balanced',max_iter=10000, solver='lbfgs', n_jobs=-1)
    score=-cross_val_score(model, Xtrain, Ytrain, cv=kf, scoring='roc_auc').mean()
    return score
study=optuna.create_study()

In [18]:
#study.optimize(objective, n_trials=50)
study.optimize(objective, n_trials=1)

[I 2020-05-11 18:50:10,385] Finished trial#1 with value: -0.7996239239077836 with parameters: {'C': 0.5660320812205071}. Best is trial#1 with value: -0.7996239239077836.


In [20]:
print(study.best_params)
print(-study.best_value)
params=study.best_params

{'C': 0.5660320812205071}
0.7996239239077836


Trial Resulted in Value : C = 0.09536298444122952

In [19]:
#model=LogisticRegression(C=0.09536298444122952, class_weight='balanced',max_iter=10000, solver='lbfgs', n_jobs=-1)
model=LogisticRegression(C=0.566, class_weight='balanced',max_iter=10000, solver='lbfgs', n_jobs=-1)

model.fit(Xtrain, Ytrain)
predictions=model.predict_proba(Xtest)[:,1]
submission['target']=predictions
submission.to_csv('simple-onehot-logisticregression-submission.csv')
submission.head()

,target
id,
300000,0.514793
300001,0.828798
300002,0.159825
300003,0.582148
300004,0.950607
